# ML localization NB-IoT: XGBoost and Random Forest

## Data preparation

In [38]:
import pandas as pd
import numpy as np


In [39]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [40]:
nr_df = pd.read_csv('/content/drive/MyDrive/localization/NBIOT_fingerprint_DB_grid.csv', sep=',')
nr_df

,Timestamp,Lat,Lon,BMAVAT1,BMAXMT1,BMAXMT2,BMAXMT3,BMBACT1,BMBACT2,BMBAKT1,...,BMZLTT1,BMZLTT3,TBTS1T1,TBTS3T1,TBTS3T2,TBTS5T1,TBTS5T2,TBTS5T3,TBTS6T1,TBTS6T2
0,1701853012,49.227134,16.575806,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1701853013,49.227133,16.575806,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1701853014,49.227133,16.575805,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1701853015,49.227132,16.575804,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1701853016,49.227131,16.575803,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28672,1701951192,49.230384,16.571568,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
28673,1701951194,49.230385,16.571568,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
28674,1701951196,49.230386,16.571568,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
28675,1701951198,49.230387,16.571568,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [41]:
nr_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28677 entries, 0 to 28676
Columns: 334 entries, Timestamp to TBTS6T2
dtypes: float64(2), int64(332)
memory usage: 73.1 MB


In [42]:
nr_df.isnull().sum()

Timestamp    0
Lat          0
Lon          0
BMAVAT1      0
BMAXMT1      0
            ..
TBTS5T1      0
TBTS5T2      0
TBTS5T3      0
TBTS6T1      0
TBTS6T2      0
Length: 334, dtype: int64

In [43]:
X = nr_df.iloc[:, 3:]
y = nr_df.iloc[:, 1:3]

In [44]:
X

,BMAVAT1,BMAXMT1,BMAXMT2,BMAXMT3,BMBACT1,BMBACT2,BMBAKT1,BMBAKT2,BMBAKT3,BMBHKT1,...,BMZLTT1,BMZLTT3,TBTS1T1,TBTS3T1,TBTS3T2,TBTS5T1,TBTS5T2,TBTS5T3,TBTS6T1,TBTS6T2
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28672,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
28673,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
28674,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
28675,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [45]:
import sklearn
from sklearn.model_selection import train_test_split

In [46]:
# Train/test split 8/2
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=36, shuffle=True, test_size=0.2)

# Confirm how the data was split
# print(X_train.shape)
# print(X_test.shape)
# print(y_train.shape)
# print(y_test.shape)

## XGBoost

In [47]:
from xgboost import XGBRegressor

xgb_model = XGBRegressor()

In [48]:
# Train the model using the training sets
xgb_model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [49]:
xgb_model.get_params()

{'objective': 'reg:squarederror',
 'base_score': None,
 'booster': None,
 'callbacks': None,
 'colsample_bylevel': None,
 'colsample_bynode': None,
 'colsample_bytree': None,
 'device': None,
 'early_stopping_rounds': None,
 'enable_categorical': False,
 'eval_metric': None,
 'feature_types': None,
 'gamma': None,
 'grow_policy': None,
 'importance_type': None,
 'interaction_constraints': None,
 'learning_rate': None,
 'max_bin': None,
 'max_cat_threshold': None,
 'max_cat_to_onehot': None,
 'max_delta_step': None,
 'max_depth': None,
 'max_leaves': None,
 'min_child_weight': None,
 'missing': nan,
 'monotone_constraints': None,
 'multi_strategy': None,
 'n_estimators': None,
 'n_jobs': None,
 'num_parallel_tree': None,
 'random_state': None,
 'reg_alpha': None,
 'reg_lambda': None,
 'sampling_method': None,
 'scale_pos_weight': None,
 'subsample': None,
 'tree_method': None,
 'validate_parameters': None,
 'verbosity': None}

In [50]:
#run the predictions on the testing data
y_xgb_pred_test = xgb_model.predict(X_test)

In [51]:
# Determine accuracy uisng 𝑅^2
from sklearn.metrics import r2_score

In [52]:
# R^2 score
score_xgb = r2_score(y_test, y_xgb_pred_test)

print("R^2 - {}%".format(round(score_xgb, 5) *100))

R^2 - 95.761%


In [53]:
# MSE and RMSE
from sklearn.metrics import mean_squared_error

In [54]:
mse_xgb = mean_squared_error(y_test, y_xgb_pred_test)
rmse_xgb = mean_squared_error(y_test, y_xgb_pred_test, squared = False)

print(mse_xgb)
print(rmse_xgb)

3.0191747809890532e-05
0.00524816352925273


In [55]:
y_test

,Lat,Lon
8662,49.198647,16.662273
22725,49.196704,16.594198
5632,49.198282,16.591177
28260,49.211263,16.571406
7377,49.210053,16.627820
...,...,...
23513,49.212285,16.562761
6768,49.200279,16.607918
15728,49.194318,16.646810
7642,49.214839,16.626566


In [56]:
y_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5736 entries, 8662 to 2564
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Lat     5736 non-null   float64
 1   Lon     5736 non-null   float64
dtypes: float64(2)
memory usage: 134.4 KB


In [57]:
y_xgb_pred_test

array([[49.197964, 16.661943],
       [49.19703 , 16.599249],
       [49.19703 , 16.609034],
       ...,
       [49.19703 , 16.643055],
       [49.20891 , 16.617199],
       [49.212933, 16.577112]], dtype=float32)

In [58]:
len(y_xgb_pred_test)

5736

In [59]:
y_test_arr = y_test.to_numpy()

In [60]:
y_test_arr

array([[49.198647, 16.662273],
       [49.196704, 16.594198],
       [49.198282, 16.591177],
       ...,
       [49.194318, 16.64681 ],
       [49.214839, 16.626566],
       [49.212614, 16.572281]])

In [61]:
from math import acos, sin, cos, radians

geo_dist_xgb = np.empty(len(y_xgb_pred_test), dtype=float)
for i in range(0, len(y_xgb_pred_test)):
  geo_dist_xgb[i] = acos(sin(radians(y_test_arr[i, 0]))*sin(radians(y_xgb_pred_test[i, 0]))+cos(radians(y_test_arr[i, 0]))*cos(radians(y_xgb_pred_test[i, 0]))*cos(radians(y_xgb_pred_test[i, 1]) - radians(y_test_arr[i, 1]))) * 6371


In [62]:
geo_dist_xgb

array([0.07966209, 0.36878195, 1.30492633, ..., 0.40660623, 0.94727801,
       0.35271344])

In [63]:
geo_dist_xgb_m = [i * 1000 for i in geo_dist_xgb]


In [74]:
# Get average of a list
def Average(lst):
    return sum(lst) / len(lst)

average_error_xgb = Average(geo_dist_xgb_m)

# Printing average of the list
print("Average error XGB [m] =", round(average_error_xgb, 5))

Average error XGB [m] = 570.22508


In [65]:
print("Max error XGB [m] = ", np.max(geo_dist_xgb_m))
print("Min error XGB [m] = ", np.min(geo_dist_xgb_m))

Max error XGB [m] =  2886.5190966049454
Min error XGB [m] =  17.340446084718934


## Random forest


In [66]:
from sklearn.ensemble import RandomForestRegressor

In [67]:
rf_model = RandomForestRegressor(n_estimators=10,random_state=10)

# Train the model using the training sets
rf_model.fit(X_train, y_train)

RandomForestRegressor(n_estimators=10, random_state=10)

In [68]:
#run the predictions on the training and testing data
y_rf_predict_test = rf_model.predict(X_test)

In [69]:
score_rf = r2_score(y_test, y_rf_predict_test)

print("R^2 - {}%".format(round(score_rf, 5) *100))

R^2 - 99.376%


In [70]:
# RMSE and MSE

mse_rf = mean_squared_error(y_test, y_rf_predict_test)
rmse_rf = mean_squared_error(y_test, y_rf_predict_test, squared = False)

print(mse_rf)
print(rmse_rf)

3.9472055305569644e-06
0.0019348185582425679


# Cross check

In [75]:
cross_check_df = pd.read_csv('/content/drive/MyDrive/localization/NBIOT_fingerprint_DB_circle.csv', sep=',')
cross_check_df

,Timestamp,Lat,Lon,BMAVAT1,BMAXMT1,BMAXMT2,BMAXMT3,BMBACT1,BMBACT2,BMBAKT1,...,BMZLTT1,BMZLTT3,TBTS1T1,TBTS3T1,TBTS3T2,TBTS5T1,TBTS5T2,TBTS5T3,TBTS6T1,TBTS6T2
0,1701439521,49.227657,16.575590,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1701439522,49.227656,16.575592,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1701439523,49.227656,16.575593,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1701439524,49.227654,16.575595,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1701439525,49.227653,16.575596,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7349,1701447326,49.197588,16.608551,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7350,1701447327,49.197582,16.608546,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7351,1701447328,49.197578,16.608546,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7352,1701447329,49.197579,16.608549,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [76]:
X_check = cross_check_df.iloc[:, 3:]
y_check = cross_check_df.iloc[:, 1:3]

## XGboost cross check

In [77]:
# Run the predictions on the cross-check data
y_xgb_cross = xgb_model.predict(X_check)

In [78]:
# R^2 score
score_xgb_cross = r2_score(y_check, y_xgb_cross)

print("R^2 - {}%".format(round(score_xgb_cross, 5) *100))

R^2 - 91.703%


In [79]:
mse_xgb_cross = mean_squared_error(y_check, y_xgb_cross)
rmse_xgb_cross = mean_squared_error(y_check, y_xgb_cross, squared = False)

print(mse_xgb_cross)
print(rmse_xgb_cross)

3.4039373749119575e-05
0.00573256348478556


In [80]:
y_check

,Lat,Lon
0,49.227657,16.575590
1,49.227656,16.575592
2,49.227656,16.575593
3,49.227654,16.575595
4,49.227653,16.575596
...,...,...
7349,49.197588,16.608551
7350,49.197582,16.608546
7351,49.197578,16.608546
7352,49.197579,16.608549


In [81]:
y_check.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7354 entries, 0 to 7353
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Lat     7354 non-null   float64
 1   Lon     7354 non-null   float64
dtypes: float64(2)
memory usage: 115.0 KB


In [82]:
y_xgb_cross

array([[49.226505, 16.57926 ],
       [49.226505, 16.57926 ],
       [49.226505, 16.57926 ],
       ...,
       [49.19703 , 16.609034],
       [49.19703 , 16.609034],
       [49.19703 , 16.609034]], dtype=float32)

In [83]:
y_check_arr = y_check.to_numpy()

In [84]:
y_check_arr

array([[49.227657, 16.57559 ],
       [49.227656, 16.575592],
       [49.227656, 16.575593],
       ...,
       [49.197578, 16.608546],
       [49.197579, 16.608549],
       [49.197576, 16.608545]])

In [85]:
# Defining the function to calculate geographical distance from coordinates
def geo_distance(true_coord, pred_coord):
  geo_dist = np.empty(len(true_coord), dtype=float)
  for i in range(0, len(true_coord)):
    geo_dist[i] = acos(sin(radians(true_coord[i, 0]))*sin(radians(pred_coord[i, 0]))+cos(radians(true_coord[i, 0]))*cos(radians(pred_coord[i, 0]))*cos(radians(pred_coord[i, 1]) - radians(true_coord[i, 1]))) * 6371
  return geo_dist

# geo_dist_xgb_check = np.empty(len(y_check), dtype=float)
# for i in range(0, len(y_check)):
#   geo_dist_xgb_check[i] = acos(sin(radians(y_check_arr[i, 0]))*sin(radians(y_xgb_pred_test[i, 0]))+cos(radians(y_test_arr[i, 0]))*cos(radians(y_xgb_pred_test[i, 0]))*cos(radians(y_xgb_pred_test[i, 1]) - radians(y_test_arr[i, 1]))) * 6371


In [86]:
geo_dist_xgb_check = geo_distance(y_check_arr, y_xgb_cross)
geo_dist_xgb_check

array([0.29567028, 0.29549124, 0.29542577, ..., 0.07057091, 0.07055801,
       0.07041518])

In [87]:
geo_dist_xgb_check_m = [i * 1000 for i in geo_dist_xgb_check]


In [88]:
average_error_xgb_check = Average(geo_dist_xgb_check_m)

# Printing average of the list
print("Average error XGB cross check [m] =", round(average_error_xgb_check, 5))

Average error XGB cross check [m] = 584.36394


In [89]:
print("Max error XGB cross check [m] = ", np.max(geo_dist_xgb_check_m))
print("Min error XGB cross check [m] = ", np.min(geo_dist_xgb_check_m))

Max error XGB cross check [m] =  3032.7562075678843
Min error XGB cross check [m] =  30.90840208372187


## RF cross check

In [90]:
#run the predictions on the training and testing data
y_rf_cross = rf_model.predict(X_check)

In [91]:
score_rf_cross = r2_score(y_check, y_rf_cross)

print("R^2 - {}%".format(round(score_rf_cross, 5) *100))

R^2 - 93.705%


In [92]:
# RMSE and MSE

mse_rf_cross = mean_squared_error(y_check, y_rf_cross)
rmse_rf_cross = mean_squared_error(y_check, y_rf_cross, squared = False)

print(mse_rf_cross)
print(rmse_rf_cross)

2.6504816598290773e-05
0.004887233956042318


In [93]:
# Distance error
geo_dist_rf_check = geo_distance(y_check_arr, y_rf_cross)

In [94]:
geo_dist_rf_check_m = [i * 1000 for i in geo_dist_rf_check]
geo_dist_rf_check_m

[31.933276542586807,
 3.7508454458474705,
 53.125948818315834,
 52.87086442877892,
 52.74337224250195,
 52.573758879364256,
 52.425520227785434,
 4.096543361103716,
 52.42956006454315,
 4.471039314764005,
 4.650860771915905,
 4.780819447388897,
 4.846351652998831,
 4.911009478764742,
 4.976638351147244,
 4.99651968030821,
 5.085909947658996,
 5.182458700625135,
 5.282362410557702,
 5.316376686096346,
 5.386271072352378,
 51.6311882529077,
 51.50508963901339,
 5.46682257005382,
 5.511155540868374,
 5.465173701800342,
 5.373706858664846,
 5.3552260237326985,
 5.328230383501383,
 5.30874247550865,
 5.2976958605483455,
 5.224892440316835,
 49.91394265983204,
 5.160674414617511,
 5.168527334830188,
 5.107130804560836,
 5.047666137067829,
 4.975732766870592,
 4.903663162791465,
 4.920176948902965,
 4.831451228329454,
 4.760037284615972,
 4.674057228890515,
 4.517164735867017,
 9.521967792790997,
 44.968376488908056,
 45.094966051768345,
 45.00724181549639,
 45.02696214343314,
 45.15368686121

In [95]:
average_error_rf_check = Average(geo_dist_rf_check_m)

# Printing average of the list
print("Average RF error cross check [m] =", round(average_error_rf_check, 5))
print("Max RF error cross check [m] =", np.max(geo_dist_rf_check_m))
print("Min RF error cross check [m] =", np.min(geo_dist_rf_check_m))

Average RF error cross check [m] = 387.81585
Max RF error cross check [m] = 3706.086694756373
Min RF error cross check [m] = 0.42456355943236246


In [96]:
geo_dist_rf = np.empty(len(y_xgb_pred_test), dtype=float)
for i in range(0, len(y_xgb_pred_test)):
  geo_dist_rf[i] = acos(sin(radians(y_test_arr[i, 0]))*sin(radians(y_rf_predict_test[i, 0]))+cos(radians(y_test_arr[i, 0]))*cos(radians(y_rf_predict_test[i, 0]))*cos(radians(y_rf_predict_test[i, 1]) - radians(y_test_arr[i, 1]))) * 6371

In [97]:
geo_dist_rf_m = [i * 1000 for i in geo_dist_rf]
geo_dist_rf_m

[414.4819553052656,
 19.7755033897625,
 5.605203818634251,
 181.97640590750285,
 62.432789603000174,
 165.3538260618883,
 238.62791163892143,
 28.84658667388607,
 93.0347514069956,
 34.75409976623722,
 641.3536445074492,
 52.67822728491846,
 37.92519599339873,
 113.22346781544549,
 27.884163259959657,
 285.43469990848416,
 12.799024630764867,
 78.62282083722033,
 91.96070218881557,
 36.241666569230404,
 29.631069179252407,
 63.094844669748184,
 305.6083881128434,
 52.341730873508034,
 19.197832761547282,
 277.42810085105,
 56.10218281317108,
 315.05013489948294,
 29.16213946830151,
 28.31014518847757,
 15.41580871611078,
 14.032130730767891,
 425.9456207468286,
 45.5552940305954,
 40.8828474772251,
 21.044609666049563,
 1473.288617884715,
 138.159499282444,
 47.515661961596145,
 240.6687510706732,
 22.091588666611127,
 0.40277635779435883,
 191.92311059637956,
 17.14837391647133,
 50.42449722145108,
 138.37020783861504,
 3.495888419937966,
 339.6260229782902,
 6.8531187045206945,
 48.9

In [98]:
average_error_rf = Average(geo_dist_rf_m)

# Printing average of the list
print("Average RF error [m] =", round(average_error_rf, 5))
print("Max RF error [m] =", np.max(geo_dist_rf_m))
print("Min RF error [m] =", np.min(geo_dist_rf_m))

Average RF error [m] = 133.50993
Max RF error [m] = 2666.9141539417824
Min RF error [m] = 0.0
